In [2]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn import tree
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder

In [ ]:
print(np.logspace(-4, 4, 9))

In [ ]:
fireWall = pd.read_csv('log2.csv')

fireWallXData = fireWall[['Source Port', 'Destination Port','NAT Source Port',
                         'NAT Destination Port','Bytes','Bytes Sent','Bytes Received','Packets',
                        'Elapsed Time (sec)','pkts_sent','pkts_received']]
fireWallyData = fireWall[['Action']]

X = fireWallXData.iloc[:,0:10]
y = fireWallyData.iloc[:,0]


print(y.shape)
print(X.shape)

#print(fireWall.shape)

FireWALLData = pd.concat([X, y], axis=1)
print(FireWALLData.shape)

newX = FireWALLData.iloc[:,0:9]
newy = FireWALLData.iloc[:,10]
FireWALLData.head()


#ncodeLabel = LabelEncoder()
#encodeLabel.fit_transform(FireWALLData.iloc[:,10])

In [ ]:
FireWALLData.head()

In [ ]:
corrMatrix = FireWALLData.corr()
fig, ax = plt.subplots(figsize=(10,10))  
sns.heatmap(corrMatrix, annot=True)


plt.show()

In [ ]:
fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = encodeLabel.fit_transform(FireWALLData['Action']),
                   colorscale = 'Rainbow',
                   cauto = True),
             dimensions = list([
             dict(range = [FireWALLData['Source Port'].min() ,FireWALLData['Source Port'].max()],
                 label = "Source Port", values = FireWALLData['Source Port']),
             dict(range = [FireWALLData['NAT Source Port'].min() ,FireWALLData['NAT Source Port'].max()],
                 label = 'NAT Source Port', values = FireWALLData['NAT Source Port']),
             dict(range = [FireWALLData['Bytes'].min() ,FireWALLData['Bytes'].max()],
                 label = 'Bytes', values = FireWALLData['Bytes']),
             dict(range = [FireWALLData['Bytes Sent'].min() ,FireWALLData['Bytes Sent'].max()],
                 label = 'Bytes Sent', values = FireWALLData['Bytes Sent']),
             dict(range = [FireWALLData['Packets'].min() ,FireWALLData['Packets'].max()],
                 label = 'Packets', values = FireWALLData['Packets']),
             dict(range = [FireWALLData['Elapsed Time (sec)'].min() ,FireWALLData['Elapsed Time (sec)'].max()],
                 label = 'Elapsed Time (sec)', values = FireWALLData['Elapsed Time (sec)']),
             dict(range = [FireWALLData['Destination Port'].min() ,FireWALLData['Destination Port'].max()],
                 label = 'Destination Port', values = FireWALLData['Destination Port']),
             dict(range = [FireWALLData['NAT Destination Port'].min() ,FireWALLData['NAT Destination Port'].max()],
                 label = 'NAT Destination Port', values = FireWALLData['NAT Destination Port'])])
    )
)
               
fig.show()

In [ ]:

from sklearn.preprocessing import LabelEncoder
encodeLabel = LabelEncoder()
newy = encodeLabel.fit_transform(newy)
print(newy)
X_train, X_test, y_train, y_test = train_test_split( newX, newy, test_size=60000, random_state=1738)


# LOGISTIC REGRESSION TRIAL ONE ON Fire Wall DATASET

In [ ]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', LogisticRegression(n_jobs = -1))])

    search_space = [
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                
                'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                'classifier__solver': ['lbfgs','saga','newton-cg','liblinear'],
                'classifier__penalty': ['none']},
        
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['newton-cg'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['liblinear'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)}
                
                ]
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

In [ ]:
TRIAL_1.cv_results_

In [ ]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_neg_log_loss']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_f1_micro']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


# LOGISTIC REGRESSION TRIAL TWO ON Fire Wall DATASET

In [ ]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', LogisticRegression(n_jobs = -1))])

    search_space = [
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                
                'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                'classifier__solver': ['lbfgs','saga','newton-cg','liblinear'],
                'classifier__penalty': ['none']},
        
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['newton-cg'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['liblinear'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)}
                
                ]
    
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

In [ ]:
TRIAL_2.cv_results_

In [ ]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_neg_log_loss']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_f1_micro']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


# LOGISTIC REGRESSION TRIAL THREE ON Fire Wall  DATASET

In [ ]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', LogisticRegression(n_jobs = -1))])

    search_space = [
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                
                'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                'classifier__solver': ['lbfgs','saga','newton-cg','liblinear'],
                'classifier__penalty': ['none']},
        
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['newton-cg'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['liblinear'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)}
                
                ]
    
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

In [ ]:
TRIAL_3.cv_results_

In [ ]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_neg_log_loss']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_f1_micro']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))


# LOGISTIC REGRESSION TRIAL FOUR ON Fire Wall DATASET

In [ ]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', LogisticRegression(n_jobs = -1))])

    search_space = [
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                
                'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                'classifier__solver': ['lbfgs','saga','newton-cg','liblinear'],
                'classifier__penalty': ['none']},
        
                {
                 
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['newton-cg'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 'classifier': [LogisticRegression(max_iter=7000,n_jobs = -1)],
                 'classifier__solver': ['liblinear'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)}
                
                ]
    
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

In [ ]:
TRIAL_4.cv_results_

In [ ]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_neg_log_loss']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_f1_micro']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


# LOGISTIC REGRESSION TRIAL FIVE ON Fire Wall DATASET

In [ ]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 25000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', LogisticRegression(n_jobs = -1))])

    search_space = [
                {
                 
                 'classifier': [LogisticRegression(n_jobs = -1)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 
                 'classifier': [LogisticRegression(n_jobs = -1)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                
                'classifier': [LogisticRegression(n_jobs = -1)],
                'classifier__solver': ['lbfgs','saga','newton-cg','liblinear'],
                'classifier__penalty': ['none']},
        
                {
                 
                 'classifier': [LogisticRegression(n_jobs = -1)],
                 'classifier__solver': ['newton-cg'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                
                {
                 'classifier': [LogisticRegression(n_jobs = -1)],
                 'classifier__solver': ['liblinear'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)}
                
                ]
    
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

In [ ]:
TRIAL_5.cv_results_

In [ ]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_neg_log_loss']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_f1_micro']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 Logistic Regression using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 Logistic Regression using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 Logistic Regression using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")
